Yue write this code at 2023/08/01 to control two motors with one ODRIVE at the same time

Try to control speed/torque/position all


In [9]:
## connection and calibration
import odrive
from odrive.enums import *
import time
import math
import matplotlib.pyplot as plt
from scipy.signal import butter,filtfilt
from IPython.display import clear_output
import numpy as np
import serial

def check(my_drive):
    if my_drive.axis1.error == 0:
        print('Everything is fine')
    else:
        print('Error!')

In [10]:
# Find a connected ODrive (this will block until you connect one)
print("finding an odrive...")
my_drive = odrive.find_any()
print("Found! and the bus voltage is " + str(my_drive.vbus_voltage) + "V")
print(my_drive.axis0.motor.current_control.i_gain,my_drive.axis1.motor.current_control.i_gain)
my_drive.axis1.motor.current_control.i_gain = 4
my_drive.axis0.motor.current_control.i_gain = 4
print(my_drive.axis0.motor.current_control.i_gain,my_drive.axis1.motor.current_control.i_gain)


# calibration and enter closed control mode
# print("starting calibration...")
# my_drive.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
# while my_drive.axis1.current_state != AXIS_STATE_IDLE:
#     time.sleep(0.1)
# check(my_drive)
# my_drive.axis1.motor.config.pre_calibrated = True
# my_drive.axis1.encoder.config.pre_calibrated =True
my_drive.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
my_drive.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL


check(my_drive)
print('*'*50)


finding an odrive...
Found! and the bus voltage is 22.440967559814453V
80.16718292236328 82.6633529663086
4.0 4.0
Everything is fine
**************************************************


In [11]:
# my_drive.axis1.motor.config.torque_constant = 1
print(my_drive.axis0.motor.config.torque_constant,my_drive.axis1.motor.config.torque_constant)
# my_drive.axis0.controller.config.pos_gain = 20
# my_drive.axis1.controller.config.pos_gain = 20
print('vel_gain and pos_gain of AXIS0 is',my_drive.axis0.controller.config.vel_gain,my_drive.axis0.controller.config.pos_gain)
print('vel_gain and pos_gain of AXIS1 is',my_drive.axis1.controller.config.vel_gain,my_drive.axis1.controller.config.pos_gain)
# my_drive.save_configuration()


1.0 1.0
vel_gain and pos_gain of AXIS0 is 8.0 20.0
vel_gain and pos_gain of AXIS1 is 8.0 20.0


In [ ]:
#torque control meantime
my_drive.axis1.controller.config.control_mode = 1
my_drive.axis0.controller.config.control_mode = 1

t0=time.monotonic()
delta=0
while delta<12:
    delta=time.monotonic()-t0
    input=delta/2
    my_drive.axis1.controller.input_torque = input
    my_drive.axis0.controller.input_torque = input
    if int(delta*100)%10 == 0:
        clear_output()
        print('INPUT CURRENT=',input)
my_drive.axis1.controller.input_torque = 0
my_drive.axis0.controller.input_torque = 0
check(my_drive)

In [ ]:
#speed control meantime
my_drive.axis1.controller.config.control_mode = 2
my_drive.axis0.controller.config.control_mode = 2
t0=time.monotonic()
delta=0
while delta<6:
    delta=time.monotonic()-t0
    input=delta/2
    my_drive.axis1.controller.input_vel = input
    my_drive.axis0.controller.input_vel = input
    if int(delta*100)%10 == 0:
        clear_output()
        print('INPUT VELOCITY=',input)
my_drive.axis1.controller.input_vel = 0
my_drive.axis0.controller.input_vel = 0
check(my_drive)

In [12]:
#position control meantime
my_drive.axis1.controller.config.control_mode = 3
my_drive.axis0.controller.config.control_mode = 3
my_drive.axis1.controller.input_pos = 0.04
my_drive.axis0.controller.input_pos = 0.4

check(my_drive)

Everything is fine


In [13]:
# sin input
t0=time.monotonic()
delta=0
my_drive.axis1.controller.config.control_mode = 3
my_drive.axis0.controller.config.control_mode = 3
FREQUENCY = 0.5
while delta<8:
    delta=time.monotonic()-t0
    pos0=0.1*math.sin(delta*2*math.pi*FREQUENCY)+0.4
    pos1=0.1*math.sin(delta*2*math.pi*FREQUENCY)+0.04
    if delta>4:
        FREQUENCY=1
    my_drive.axis0.controller.input_pos=pos0
    my_drive.axis1.controller.input_pos=pos1
    if int(delta*100)%10==0:
        clear_output()
        print('POSITION INPUT \nAXIS0:{:.2f} degree \nAXIS1:{:.2f} degree'.format(pos0*180/math.pi,pos1*180/math.pi))

POSITION INPUT 
AXIS0:22.92 degree 
AXIS1:2.29 degree


In [7]:
check(my_drive)

Everything is fine
